In [4]:
import requests
import pandas as pd

def get_basic_income_amounts(policy_id):
    # API endpoint URL
    api_url = f"https://api.policyengine.org/us/policy/{policy_id}"

    # Making a GET request to the API
    response = requests.get(api_url)

    if response.status_code == 200:
        # Parsing the JSON response
        policy_data = response.json()

        # Extracting and renaming the basic income amounts
        basic_income_amounts = {}
        age_group_mapping = {
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[0].amount": "ubi_0_5",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[1].amount": "ubi_6_17",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[2].amount": "ubi_18_64",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[4].amount": "ubi_65_plus",
            "gov.contrib.ubi_center.basic_income.amount.person.disability": "ubi_disability"
        }

        for key, value in policy_data["result"]["policy_json"].items():
            if key in age_group_mapping:
                # Assuming the amount is the first value in the dictionary
                amount = next(iter(value.values()))
                basic_income_amounts[age_group_mapping[key]] = amount
        return basic_income_amounts
    else:
        return None

# Read the CSV file into a DataFrame
df = pd.read_csv('losses_by_rate.csv')

# Fetch and add basic income amounts for each reform ID
for i, row in df.iterrows():
    reform_id = row['api_id']
    amounts = get_basic_income_amounts(reform_id)
    if amounts:
        for column_name, amount in amounts.items():
            df.loc[i, column_name] = amount

print(df)


    flat_tax_rate      loss  api_id  Labour supply responses       ubi_0_5  \
0            0.20  0.121484   42518  Labour supply responses   3784.780920   
1            0.21  0.108601   42519  Labour supply responses   4202.951551   
2            0.22  0.096196   42520  Labour supply responses   4102.708659   
3            0.23  0.084663   42521  Labour supply responses   3761.955950   
4            0.24  0.074044   42522  Labour supply responses   3976.256888   
5            0.25  0.064599   42523  Labour supply responses   3835.308240   
6            0.26  0.056402   42524  Labour supply responses   3334.781121   
7            0.27  0.050775   42525  Labour supply responses   3017.174890   
8            0.28  0.047745   42526  Labour supply responses   3372.771916   
9            0.29  0.046852   42527  Labour supply responses   3796.969379   
10           0.30  0.047145   42528  Labour supply responses   3985.542299   
11           0.31  0.048338   42529  Labour supply responses   3

In [6]:
import plotly.express as px

# Melt the DataFrame to make it suitable for Plotly
melted_df = df.melt(id_vars=['flat_tax_rate'], 
                    value_vars=['ubi_0_5', 'ubi_6_17', 'ubi_18_64', 'ubi_65_plus', 'ubi_disability'],
                    var_name='UBI Category', value_name='Amount')

# Create the plot
fig = px.line(melted_df, x='flat_tax_rate', y='Amount', color='UBI Category',
              labels={'flat_tax_rate': 'Flat Tax Rate', 'Amount': 'UBI Amount'},
              title='UBI Amounts by Flat Tax Rate')

# Show the plot
fig.show()